In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import OneClassSVM
dc_evt = pd.read_csv('data/dc_evt_modified2.csv', low_memory=False)
dc_file = pd.read_csv('data/dc_file_modified2.csv', low_memory=False)
binary_col_file = [col for col in dc_file.columns if set(dc_file[col])|{0, 1} == {0, 1}]
binary_col_evt = [col for col in dc_evt.columns if set(dc_evt[col])|{0, 1} == {0, 1}]
bin_file_ag = dc_file[['inode']+binary_col_file].groupby('inode').sum()
bin_evt_ag = dc_evt[['inode']+binary_col_evt].groupby('inode').sum()
model = OneClassSVM(kernel='rbf', nu=0.0004998348)
model.fit(bin_file_ag)
y_pred = model.predict(bin_file_ag)
outliers = np.where(y_pred == -1)[0]
num_outliers = len(outliers)

outliers, num_outliers, len(bin_evt_ag)